# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [34]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/caroline/Documents/TU/Robo/myenv/lib/python3.12/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [35]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['HeadBack', 'Stand', 'Left', 'Sit', 'Back', 'StandInit', 'Right', 'Crouch', 'Belly', 'Frog', 'Knee']


In [36]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [37]:
# load all the data
all_data = []
all_target = []
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print('Total number of data:', len(all_data))

Total number of data: 222


In [39]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [40]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [43]:
# Use the permutation indices to reorder all_data and all_target
training_data = [all_data[i] for i in permutation[:n_training_data]]
training_target = [all_target[i] for i in permutation[:n_training_data]]
testing_data = [all_data[i] for i in permutation[n_training_data:]]
testing_target = [all_target[i] for i in permutation[n_training_data:]]

# Ensure data is in the correct format (2D arrays)
training_data = np.array(training_data)
training_target = np.array(training_target)
testing_data = np.array(testing_data)
testing_target = np.array(testing_target)


clf.fit(training_data, training_target)
print(clf.get_params())

{'C': 100.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


### predicting

In [45]:
# Predict for the last entry in all_data
clf.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)

In [46]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [48]:
expected = training_target
predicted = clf.predict(training_data)
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00        18
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        36
           6       1.00      0.86      0.92         7
           7       0.95      1.00      0.98        21
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00         8

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [49]:
expected = testing_target
predicted = clf.predict(testing_data)

# Evaluate the predictions
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00        16
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         2

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 2  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [57]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [59]:
# Loading the model
with open(ROBOT_POSE_CLF, 'rb') as file:
    clf2 = pickle.load(file)
clf2.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)